# pandas 시계열 · 시간 타입 완전 정리 노트

이 노트북은 다음 내용을 **설명 + 실습 코드** 형태로 정리한 것입니다.

- `Timestamp`, `DatetimeIndex`, `datetime64[ns]`
- `Timedelta`, `TimedeltaIndex`, `timedelta64[ns]`
- `DateOffset` 계열 (`MonthEnd`, `BDay` 등)
- `Period`, `PeriodIndex`
- `pd.to_datetime`, `pd.to_timedelta`, `.dt` 접근자
- Time Zone(`tz_localize`, `tz_convert`), `NaT`
- **추가 실습용 셀** 포함

In [89]:
import pandas as pd
import numpy as np

pd.__version__

'2.3.3'

- 파이썬 표준 라이브러리 : datetime

    ```python
    import datetime

    dt = datetime.datetime(2024, 1, 1, 12, 30)
    ```

## 1. Timestamp / DatetimeIndex / `datetime64[ns]`

- pd.Timestamp()
  - 입력을 날짜/시간 표현으로 변환
  - Scalar 만 입력, 반환

- pd.to_datetime()
  - 입력을 날짜/시간 표현으로 변환
  - 이미 있는 포맷을 날짜로 바꿈

- pd.date_range() 
  - DatetimeIndex 를 반환
  - 정해진 규칙에 따라 생성

### 1-1. `pd.Timestamp`: 한 개의 날짜·시간

`Timestamp`는 pandas 버전의 `datetime.datetime`이라고 생각하면 됩니다. 한 시점을 나타냅니다.


In [90]:
ts = pd.Timestamp('2025-01-15 13:30:00')
print(ts)
print(type(ts))

print("year:", ts.year)
print("month:", ts.month)
print("day:", ts.day)
print("hour:", ts.hour)
print("minute:", ts.minute)
print("second:", ts.second)
print("day_name:", ts.day_name())

2025-01-15 13:30:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
year: 2025
month: 1
day: 15
hour: 13
minute: 30
second: 0
day_name: Wednesday


### 1-2. `DatetimeIndex`: 여러 시점으로 구성된 인덱스

시계열 데이터에서는 날짜/시간을 인덱스로 두는 경우가 많습니다.


In [91]:
idx = pd.date_range('2025-01-01', periods=5, freq='D')
idx

DatetimeIndex(['2025-01-01', '2025-01-02', '2025-01-03', '2025-01-04',
               '2025-01-05'],
              dtype='datetime64[ns]', freq='D')

In [92]:
s = pd.Series(range(5), index=idx)
s

2025-01-01    0
2025-01-02    1
2025-01-03    2
2025-01-04    3
2025-01-05    4
Freq: D, dtype: int64

In [93]:
# 날짜 슬라이싱 예시
s['2025-01-02':'2025-01-04']

2025-01-02    1
2025-01-03    2
2025-01-04    3
Freq: D, dtype: int64

### 1-3. `datetime64[ns]` dtype과 `Timestamp` 원소

- pd.to_datetime()
  - 입력을 날짜/시간 표현으로 변환

- Series 입력시 배열 전체의 저장형식은 `datetime64[ns]` dtype 으로 바뀌고, 각 원소는 `Timestamp` 객체입니다.
  - 배열에 저장된 datetime64[ns] 스칼라 값을, 원소 단위로 접근할 때 Timestamp 객체로 감싸서 제공한다.


In [94]:
dates = pd.Series(['2025-01-01', '2025-02-01'])
s_dt = pd.to_datetime(dates)

print("dtype:", dates.dtype)
print("dtype:", s_dt.dtype)
print("첫 원소 클래스 타입:", type(s_dt.iloc[0]))
s_dt

dtype: object
dtype: datetime64[ns]
첫 원소 클래스 타입: <class 'pandas._libs.tslibs.timestamps.Timestamp'>


0   2025-01-01
1   2025-02-01
dtype: datetime64[ns]

- 컴퓨터는 날짜/시간을 인간처럼 이해하지 못한다.
- 파일에 저장된 날짜/시간은 보통 문자열(예: "2025-01-01 09:00:00")이거나, 경우에 따라 정수[ns]( nanosecond $10^{-9}$)
- 문자열은 단순한 글자라서 날짜 연산이 불가능하다.
- 그래서 datetime(및 pandas의 Timestamp) 같은 라이브러리가 필요하다.
- 이 변환이 거의 매번 반복되는 “필수 전처리”다.

#### ✅ 추가 실습 1

1. 본인이 원하는 날짜 문자열 리스트를 만들어 `pd.to_datetime`으로 변환해 보세요.  
2. `pd.date_range`를 사용해, 2024년 1분기(1~3월) 동안 **주말만** 포함하는 `DatetimeIndex`를 만들어 보세요.  
3. `Series`에 임의의 값을 넣고, 특정 날짜 구간만 슬라이싱해 보세요.

> 아래 셀에 직접 코드를 작성해 보세요.


In [95]:
# TODO: 추가 실습 1을 이 셀에서 진행해 보세요.

# 1) 자유롭게 날짜 문자열 리스트를 만들고 to_datetime으로 변환
# 2) 주말만 포함하는 DatetimeIndex 만들기 (freq='W-SAT', 'W-SUN' 등 활용)
# 3) 날짜 슬라이싱 실습


print("\n추가 실습 1:")
# 1) 자유롭게 날짜 문자열 리스트를 만들고 to_datetime으로 변환
date_strings = ['2025-03-01', '2025-03-15', '2025-03-30']
converted_dates = pd.to_datetime(date_strings)
print("변환된 날짜들:")
print(converted_dates)
print("\n")
# 2) 주말만 포함하는 DatetimeIndex 만들기 (freq='W-SAT', 'W-SUN' 등 활용)
weekend_idx = pd.date_range('2025-03-01', periods=8, freq='W-SAT')
print("주말만 포함하는 DatetimeIndex:")
print(weekend_idx)
print("\n")
# 3) 날짜 슬라이싱 실습
data = pd.Series(range(10), index=pd.date_range('2025-03-01', periods=10, freq='D'))
sliced_data = data['2025-03-03':'2025-03-07']
print("날짜 슬라이싱 결과:")
print(sliced_data)




추가 실습 1:
변환된 날짜들:
DatetimeIndex(['2025-03-01', '2025-03-15', '2025-03-30'], dtype='datetime64[ns]', freq=None)


주말만 포함하는 DatetimeIndex:
DatetimeIndex(['2025-03-01', '2025-03-08', '2025-03-15', '2025-03-22',
               '2025-03-29', '2025-04-05', '2025-04-12', '2025-04-19'],
              dtype='datetime64[ns]', freq='W-SAT')


날짜 슬라이싱 결과:
2025-03-03    2
2025-03-04    3
2025-03-05    4
2025-03-06    5
2025-03-07    6
Freq: D, dtype: int64


## 2. Timedelta / TimedeltaIndex / `timedelta64[ns]`

- pd.Timedelta()
  - 단일 기간 생성

- pd.to_timedelta()
  - 입력을 날짜/시간 간격 표현(Timedelta)으로 변환
  - 이미 있는 포맷을 간격으로 바꿈
  - scalar => Timedelta
  - 배열 => TimedeltaIndex

- pd.timedelta_range()
  - TimedeltaIndex 를 반환
  - 정해진 규칙에 따라 생성

### 2-1. `pd.Timedelta`: 두 시점 간의 시간 차이(기간)

두 `Timestamp`를 빼면 `Timedelta`가 됩니다. 순수한 **기간(시간 차)** 입니다.


In [96]:
start = pd.Timestamp('2025-01-01 09:00')
end   = pd.Timestamp('2025-01-01 15:30')

delta = end - start
print(delta)
print(type(delta))

0 days 06:30:00
<class 'pandas._libs.tslibs.timedeltas.Timedelta'>


In [97]:
# Timedelta 직접 생성 예시
d1 = pd.Timedelta(days=3, hours=5)
d2 = pd.Timedelta('2 days 03:30:00')
d3 = pd.Timedelta('90min')

d1, d2, d3

(Timedelta('3 days 05:00:00'),
 Timedelta('2 days 03:30:00'),
 Timedelta('0 days 01:30:00'))

In [98]:
print("days:", d1.days) # 기간
print("seconds:", d1.seconds) # 시간 부분을 초로 환산
print("total seconds:", d1.total_seconds()) # 기간 + 시간을 초로 환산

days: 3
seconds: 18000
total seconds: 277200.0


```
pd.Timedelta("2 days 03:30:00")
pd.Timedelta("3 day")
pd.Timedelta("4 hours")
pd.Timedelta("90min")
pd.Timedelta("1h 20m 5s")
pd.Timedelta("500ms")
pd.Timedelta("200us")
pd.Timedelta("300ns")

pd.Timedelta("03:30:00")      # 3시간 30분
pd.Timedelta("1:02:03")       # 1시간 2분 3초
pd.Timedelta("00:00:00.123")  # 소수초

pd.Timedelta(np.timedelta64(3, "D"))
pd.Timedelta(np.timedelta64(90, "m"))

pd.to_timedelta(["1 day", "2 days 03:00:00", "90min"])
pd.to_timedelta([1, 2, 3], unit="h")
```

### 2-2. Timedelta를 사용하는 대표적인 이유

1. **최근 N일/시간 필터링**
2. **이벤트 간 시간 간격 분석**
3. **기준 시점에서 정확히 N시간/일 뒤 시점 계산**

- <font color=orange>날짜/시간을 숫자처럼 더하고 빼는 연산</font>을 가능하게 한다


np.random.randn(60) : 평균 0 주변에서 위아래로 튀는 값들(점들이 랜덤하게 흩어짐)

np.random.randn(60).cumsum() : 랜덤한 변화량을 매일 누적 → “가격/지수/잔액”처럼 추세·연속성

In [99]:
# 예시: 최근 30일 필터링
dates = pd.date_range('2025-01-01', periods=60, freq='D') # DatetimeIndex 생성
values = np.random.randn(60).cumsum()
df = pd.DataFrame({'date': dates, 'value': values}).set_index('date')

today = df.index.max()
recent_30d = df[df.index > today - pd.Timedelta(days=30)]

print("전체 길이:", len(df))
print("최근 30일 길이:", len(recent_30d))
recent_30d.head()

전체 길이: 60
최근 30일 길이: 30


,value
date,
2025-01-31,-4.124012
2025-02-01,-4.217069
2025-02-02,-5.764672
2025-02-03,-6.518275
2025-02-04,-6.340511


In [100]:
# 예시: 이벤트 간 간격 분석
ts_idx = pd.date_range('2025-01-01 09:00', periods=5, freq='2H') # DatetimeIndex 생성
df_gap = pd.DataFrame({'timestamp': ts_idx, 'value': range(5)})
df_gap['delta'] = df_gap['timestamp'].diff()
df_gap['delta_sec'] = df_gap['delta'].dt.total_seconds()
df_gap

/var/folders/y0/mx7tr20s5zg66f187z9mfcbw0000gn/T/ipykernel_50421/3777188486.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  ts_idx = pd.date_range('2025-01-01 09:00', periods=5, freq='2H') # DatetimeIndex 생성


,timestamp,value,delta,delta_sec
0,2025-01-01 09:00:00,0,NaT,NaN
1,2025-01-01 11:00:00,1,0 days 02:00:00,7200.0
2,2025-01-01 13:00:00,2,0 days 02:00:00,7200.0
3,2025-01-01 15:00:00,3,0 days 02:00:00,7200.0
4,2025-01-01 17:00:00,4,0 days 02:00:00,7200.0


### 2-3. `TimedeltaIndex`: 인덱스 자체가 "경과 시간"일 때

- pd.to_timedelta()
  - 입력을 날짜/시간 간격 표현(Timedelta)으로 변환
  - scalar => Timedelta
  - 배열 => TimedeltaIndex

실험, 시뮬레이션, 센서 데이터 등에서는 절대 시각보다 **시작 후 몇 초/분이 지났는지**가 더 의미 있을 수 있습니다.


In [101]:
td_index = pd.to_timedelta([0, 5, 10, 15], unit='m') # TimedeltaIndex 생성
s_td_idx = pd.Series([10, 20, 30, 40], index=td_index)
s_td_idx

0 days 00:00:00    10
0 days 00:05:00    20
0 days 00:10:00    30
0 days 00:15:00    40
dtype: int64

`DatetimeIndex`에서 기준 시점을 빼서 `TimedeltaIndex`로 만드는 방식도 자주 사용합니다.


In [102]:
idx = pd.date_range('2025-01-01 09:00', periods=4, freq='H') # DatetimeIndex 생성
base = idx[0]
td_idx2 = idx - base  # TimedeltaIndex
print("TimeStamp:", idx)
print("Timedelta:", td_idx2)

TimeStamp: DatetimeIndex(['2025-01-01 09:00:00', '2025-01-01 10:00:00',
               '2025-01-01 11:00:00', '2025-01-01 12:00:00'],
              dtype='datetime64[ns]', freq='h')
Timedelta: TimedeltaIndex(['0 days 00:00:00', '0 days 01:00:00', '0 days 02:00:00',
                '0 days 03:00:00'],
               dtype='timedelta64[ns]', freq='h')


/var/folders/y0/mx7tr20s5zg66f187z9mfcbw0000gn/T/ipykernel_50421/56398132.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  idx = pd.date_range('2025-01-01 09:00', periods=4, freq='H') # DatetimeIndex 생성


### 2-4. `TimedeltaIndex`

- pd.timedelta_range()
  - 규칙 생성 → TimedeltaIndex

In [103]:
pd.timedelta_range("0 days", periods=5, freq="6H")

/var/folders/y0/mx7tr20s5zg66f187z9mfcbw0000gn/T/ipykernel_50421/4224963611.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  pd.timedelta_range("0 days", periods=5, freq="6H")


TimedeltaIndex(['0 days 00:00:00', '0 days 06:00:00', '0 days 12:00:00',
                '0 days 18:00:00', '1 days 00:00:00'],
               dtype='timedelta64[ns]', freq='6h')

#### ✅ 추가 실습 2

1. `pd.date_range`로 1분 간격의 타임스탬프 10개를 만들고,  
   첫 번째 시점을 기준으로 한 `TimedeltaIndex`(경과 시간 인덱스)를 만들어 보세요.
2. 가상의 센서 데이터(예: 온도)를 만들어, **시작 후 0분 ~ 9분**을 인덱스로 사용하는 Series를 만들어 보세요.
3. `diff()`와 `dt.total_seconds()`를 활용해, 이벤트 간 간격이 60초 이상인 구간만 골라보세요.
4. 예시를 위해 일부 이벤트 누락(간격이 120초 생기게)

> 아래 셀에서 직접 실습해 보세요.


In [140]:
# 1) 1분 간격 타임스탬프 10개 + (첫 시점 기준) TimedeltaIndex 만들기
ts = pd.date_range("2025-01-01 09:00:00", periods=10, freq="min")  # 1분 간격 10개
elapsed = ts - ts[0]  # 첫 시점 기준 경과시간 (TimedeltaIndex)
print("[1] timestamps:")
print(ts)
print("\n[1] elapsed TimedeltaIndex:")
print(elapsed)
# 2) 가상의 센서 데이터(예: 온도) + 인덱스를 0~9분(경과시간)으로 Series 만들기
np.random.seed(0)
temp = 20 + np.random.randn(10).cumsum() * 0.1
sensor = pd.Series(temp, index=elapsed, name="temperature")
print("\n[2] sensor series (index=elapsed):")
print(sensor)
# 3) diff() + dt.total_seconds()로 이벤트 간격이 60초 이상인 구간만 고르기
# 예시를 위해 일부 이벤트 누락(간격이 120초 생기게)
events = ts.delete([3, 7])  # 09:03, 09:07 제거 → 2분 간격 구간 발생
events
gaps = events.to_series().diff()             # Timedelta Series
gaps
gap_seconds = gaps.dt.total_seconds()        # 초 단위로 변환
gap_seconds
over_60s = gap_seconds[gap_seconds >= 60]    # 60초 이상만 필터링
over_60s
print("\n\n events (with missing points):")
print(events)
print("\n\n gap seconds:")
print(gap_seconds)
print("\n\n gaps >= 60 seconds:")
print(over_60s)

[1] timestamps:
DatetimeIndex(['2025-01-01 09:00:00', '2025-01-01 09:01:00',
               '2025-01-01 09:02:00', '2025-01-01 09:03:00',
               '2025-01-01 09:04:00', '2025-01-01 09:05:00',
               '2025-01-01 09:06:00', '2025-01-01 09:07:00',
               '2025-01-01 09:08:00', '2025-01-01 09:09:00'],
              dtype='datetime64[ns]', freq='min')

[1] elapsed TimedeltaIndex:
TimedeltaIndex(['0 days 00:00:00', '0 days 00:01:00', '0 days 00:02:00',
                '0 days 00:03:00', '0 days 00:04:00', '0 days 00:05:00',
                '0 days 00:06:00', '0 days 00:07:00', '0 days 00:08:00',
                '0 days 00:09:00'],
               dtype='timedelta64[ns]', freq='min')

[2] sensor series (index=elapsed):
0 days 00:00:00    20.176405
0 days 00:01:00    20.216421
0 days 00:02:00    20.314295
0 days 00:03:00    20.538384
0 days 00:04:00    20.725140
0 days 00:05:00    20.627412
0 days 00:06:00    20.722421
0 days 00:07:00    20.707285
0 days 00:08:00    20.69

In [146]:
# 1) 1분 간격 DatetimeIndex 생성 → TimedeltaIndex로
print("1분 간격 DatetimeIndex 생성 → TimedeltaIndex로")
ts_idx = pd.date_range('2025-12-16 09:00', periods=10, freq='T')
print(ts_idx)
base_time = ts_idx[0]
td_idx = ts_idx - base_time
print("TimedeltaIndex:")
print(td_idx)
print("="*80)

# 2) 가상의 센서 데이터 Series 생성
print("가상의 센서 데이터 Series 생성")
sensor_data_temp = pd.Series(np.random.randn(10) * 10 + 20, index=td_idx) ## 랜덤 온도 생성
print("센서 데이터 Series:")
print(sensor_data_temp)
print("="*80)

# 3) 이벤트 간 간격이 60초 이상인 구간 골라내기
#  예시를 위해 일부 이벤트 누락(간격이 120초 생기게)
events = ts_idx.delete([3, 7]) # # 09:03, 09:07 제거 → 2분 간격 구간 발생
print("이벤트 간 간격이 60초 이상인 구간 골라내기")
gaps = events.to_series().diff()             # Timedelta Series
gap_seconds = gaps.dt.total_seconds()        # 초 단위로 변환
over_60s = gap_seconds[gap_seconds >= 60]    # 60초
print("60초 이상인 구간:")
print(over_60s)



1분 간격 DatetimeIndex 생성 → TimedeltaIndex로
DatetimeIndex(['2025-12-16 09:00:00', '2025-12-16 09:01:00',
               '2025-12-16 09:02:00', '2025-12-16 09:03:00',
               '2025-12-16 09:04:00', '2025-12-16 09:05:00',
               '2025-12-16 09:06:00', '2025-12-16 09:07:00',
               '2025-12-16 09:08:00', '2025-12-16 09:09:00'],
              dtype='datetime64[ns]', freq='min')
TimedeltaIndex:
TimedeltaIndex(['0 days 00:00:00', '0 days 00:01:00', '0 days 00:02:00',
                '0 days 00:03:00', '0 days 00:04:00', '0 days 00:05:00',
                '0 days 00:06:00', '0 days 00:07:00', '0 days 00:08:00',
                '0 days 00:09:00'],
               dtype='timedelta64[ns]', freq='min')
가상의 센서 데이터 Series 생성
센서 데이터 Series:
0 days 00:00:00    13.275396
0 days 00:01:00    16.404468
0 days 00:02:00    11.868537
0 days 00:03:00     2.737174
0 days 00:04:00    21.774261
0 days 00:05:00    15.982191
0 days 00:06:00     3.698017
0 days 00:07:00    24.627823
0 days 00:08

/var/folders/y0/mx7tr20s5zg66f187z9mfcbw0000gn/T/ipykernel_50421/2644817388.py:3: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  ts_idx = pd.date_range('2025-12-16 09:00', periods=10, freq='T')


## 3. DateOffset: 달력 규칙 기반 이동

`Timedelta`는 **정확히 며칠/몇 초**처럼 물리 시간에 초점을 맞추지만,  
`DateOffset` 계열은 **달력(calendar) 규칙**에 맞춰 움직입니다.

대표적인 예:
- `MonthEnd` : 월말로 이동
- `MonthBegin` : 월초로 이동
- `BDay` : 영업일(주말 제외) 단위 이동
- `QuarterEnd`, `YearEnd` 등


- 기준점(Anchor) 기반 이동
  - MonthEnd/MonthBegin, QuarterEnd/QuarterBegin, YearEnd/YearBegin, BMonthEnd/BMonthBegin, Week, ...
  - 0 : 해당 기준점으로 이동
  - 1 ~ : n 번째 기준점으로 이동

- 고정길이(Tick) 기반 이동
  - Day, Hour, Minute, Second, Milli, Micro, Nano, Bday, ...
  - 길이만큼 이동

In [105]:
from pandas.tseries.offsets import BDay, MonthEnd, MonthBegin, QuarterEnd

ts = pd.Timestamp('2025-01-30')
print("기준 시점:", ts)

print("이번 달 말일:", ts + MonthEnd(0)) 
print("다음 달 말일:", ts + MonthEnd(1))
print("다다음 달 말일:", ts + MonthEnd(2))

기준 시점: 2025-01-30 00:00:00
이번 달 말일: 2025-01-31 00:00:00
다음 달 말일: 2025-01-31 00:00:00
다다음 달 말일: 2025-02-28 00:00:00


In [106]:
d = pd.Timestamp('2025-01-10')  # 금요일
print("기준:", d)
print("영업일 1일 후:", d + BDay(1))  # 주말 건너뛰기
print("영업일 3일 후:", d + BDay(3))

기준: 2025-01-10 00:00:00
영업일 1일 후: 2025-01-13 00:00:00
영업일 3일 후: 2025-01-15 00:00:00


### 3-1. 월말/분기말 리밸런싱 날짜 구하기 예시

금융 데이터에서 자주 나오는 패턴: **월말/분기말에만 작업/리밸런싱**.


In [107]:
# 2025년 전체 날짜 인덱스
idx_2025 = pd.date_range('2025-01-01', '2025-12-31', freq='D') # DatetimeIndex 생성
df_2025 = pd.DataFrame(index=idx_2025, data={'value': np.random.randn(len(idx_2025))})

print(len(df_2025))

# 각 날짜를 해당 월말로 매핑
# df_2025 = df_2025.sort_index()
df_2025['month_end'] = df_2025.index + MonthEnd(0)

# 월말 별로 마지막 행만 추출 (리밸런싱 날짜)
rebalance_dates = df_2025.groupby('month_end').tail(1)
rebalance_dates

365


,value,month_end
2025-01-31,0.797562,2025-01-31
2025-02-28,0.839888,2025-02-28
2025-03-31,0.678052,2025-03-31
2025-04-30,0.245523,2025-04-30
2025-05-31,-0.726039,2025-05-31
2025-06-30,-1.940983,2025-06-30
2025-07-31,0.104639,2025-07-31
2025-08-31,-0.701558,2025-08-31
2025-09-30,0.601436,2025-09-30
2025-10-31,1.547182,2025-10-31


In [108]:
rebalance_dates[['value']]

,value
2025-01-31,0.797562
2025-02-28,0.839888
2025-03-31,0.678052
2025-04-30,0.245523
2025-05-31,-0.726039
2025-06-30,-1.940983
2025-07-31,0.104639
2025-08-31,-0.701558
2025-09-30,0.601436
2025-10-31,1.547182


#### ✅ 추가 실습 3

1. 2024년 전체 영업일(`freq='B'`)에 대해 임의의 주가 시계열을 만들고,  
   **각 분기말(`QuarterEnd`)마다 리밸런싱 날짜**를 구해보세요.
2. `BDay`를 사용해, 특정 날짜(예: 2024-01-10) 기준 **영업일 기준 -3일, +5일** 시점을 구해보세요.

> 아래 셀에서 직접 실습해 보세요.


In [ ]:
# TODO: 추가 실습 3을 이 셀에서 진행해 보세요.

# 1) 2024년 영업일 시퀀스 + 임의의 주가 -> 분기말 리밸런싱 날짜 추출
# 2) BDay를 활용한 영업일 기준 날짜 이동

print("\n추가 실습 3:")
# 1) 2024년 영업일 시퀀스 + 임의의 주가 -> 분기말 리밸런싱 날짜 추출
idx_2024 = pd.date_range('2024-01-01', '2024-12-31', freq='B') # 2024년 영업일 시퀀스 생성
df_2024 = pd.DataFrame(index=idx_2024, data={'price': np.random.randn(len



## 4. Period / PeriodIndex: 구간(기간) 타입

`Timestamp`는 특정 시점,  
`Period`는 **월/분기/연 같은 한 구간**입니다.

- 구간/범위를 대표하는 Label 처럼 사용
  - 구간에서 시점을 다양하게 추출하여 사용

- pd.Period()
  - 입력을 단일 구간 표현으로 변환

- pd.to_period()
  - Timestamp 입력을 구간 표현으로 변환

- pd.period_range()
  - PeriodIndex 를 반환
  - 정해진 규칙에 따라 생성

In [110]:
p = pd.Period('2025-01', freq='M')  # 2025년 1월이라는 "한 달"
print(p)
print(type(p))

2025-01
<class 'pandas._libs.tslibs.period.Period'>


In [111]:
pi = pd.period_range('2025-01', periods=4, freq='Q') # PeriodIndex 생성
pi

PeriodIndex(['2025Q1', '2025Q2', '2025Q3', '2025Q4'], dtype='period[Q-DEC]')

`DatetimeIndex`를 `to_period`로 바꾸면, 월/분기 단위 분석이 쉬워집니다.


In [112]:
dates = pd.date_range('2025-01-01', '2025-06-30', freq='D') # DatetimeIndex 생성
sales = np.random.randint(100, 1000, size=len(dates))
df_sales = pd.DataFrame({'date': dates, 'sales': sales}).set_index('date')

# 월 단위 PeriodIndex로 변환
df_sales['month'] = df_sales.index.to_period('M')
monthly_sales = df_sales.groupby('month')['sales'].sum()
monthly_sales

month
2025-01    14577
2025-02    14914
2025-03    17541
2025-04    17184
2025-05    17161
2025-06    18161
Freq: M, Name: sales, dtype: int64

In [113]:
df_sales.head()

,sales,month
date,,
2025-01-01,551,2025-01
2025-01-02,378,2025-01
2025-01-03,427,2025-01
2025-01-04,283,2025-01
2025-01-05,327,2025-01


#### ✅ 추가 실습 4

1. 임의의 일별 매출 데이터로부터 **분기(Quarter) 단위 매출 합계**를 구해보세요.  
2. `to_period('Q')`와 `groupby`를 활용해 보세요.
3. 월별 평균 매출, 분기별 최대 매출 등 다양한 조합을 실험해 보세요.

> 아래 셀에서 직접 실습해 보세요.


In [114]:
# TODO: 추가 실습 4를 이 셀에서 진행해 보세요.

# 1) 일별 매출 -> 분기별 매출 합계
# 2) 월별 평균, 분기별 최대 등



## 5. `pd.to_datetime`, `pd.to_timedelta`, `.dt` 접근자

문자열/숫자/타입이 뒤섞인 열을 **날짜/시간 타입으로 정리**하는 작업은 필수입니다.


In [115]:
raw_dates = pd.Series(['2025-01-01', '2025/02/01', '2025-03-01 12:00'])
s_dt = pd.to_datetime(raw_dates, format='mixed')
s_dt

0   2025-01-01 00:00:00
1   2025-02-01 00:00:00
2   2025-03-01 12:00:00
dtype: datetime64[ns]

In [116]:
raw_durations = pd.Series(['1D', '2D 03:00:00', '00:30:00'])
s_td = pd.to_timedelta(raw_durations)
print('raw_durations:\n',raw_durations)
print('\ns_td:\n',s_td)

raw_durations:
 0             1D
1    2D 03:00:00
2       00:30:00
dtype: object

s_td:
 0   1 days 00:00:00
1   2 days 03:00:00
2   0 days 00:30:00
dtype: timedelta64[ns]


### `.dt` 접근자: 날짜/시간/기간 속성 꺼내기

- Series 에서 원소별 datetime 기능을 쓰기 위한 접근자
  - 원소에 대해 같은 작업을 적용하여 새로운 Series 반환


In [117]:
s_dt = pd.to_datetime(pd.Series(['2024-01-01 12:34:56', '2024-02-03 09:10:11']))

print("year:", s_dt.dt.year.tolist())
print("month:", s_dt.dt.month.tolist())
print("day:", s_dt.dt.day.tolist())
print("weekday:", s_dt.dt.dayofweek.tolist())
print("day_name:", s_dt.dt.day_name().tolist())

year: [2024, 2024]
month: [1, 2]
day: [1, 3]
weekday: [0, 5]
day_name: ['Monday', 'Saturday']


In [118]:
s_td = pd.to_timedelta(pd.Series(['1 days 02:30:00', '0 days 05:00:00']))

print("days:", s_td.dt.days.tolist())
print("seconds:", s_td.dt.seconds.tolist())
print("total_seconds:", s_td.dt.total_seconds().tolist())

days: [1, 0]
seconds: [9000, 18000]
total_seconds: [95400.0, 18000.0]


#### ✅ 추가 실습 5

1. 문자열로 된 날짜/시간 열을 직접 만들어 `pd.to_datetime`으로 변환해보세요.  
   - 형식이 조금 다른 경우(`2024-01-01`, `2024/01/02 13:00`)도 섞어보세요.
2. `.dt.year`, `.dt.month`, `.dt.dayofweek`, `.dt.hour` 등을 이용해 파생 변수를 만들어보세요.
3. `Timedelta`를 사용해서 **이벤트 간 간격이 1시간 이상인 행만 필터링**해 보세요.

> 아래 셀에서 직접 실습해 보세요.


In [119]:
# TODO: 추가 실습 5를 이 셀에서 진행해 보세요.

# 1) 문자열 날짜/시간 -> to_datetime
# 2) .dt 접근자로 파생 변수 만들기
# 3) Timedelta를 활용한 간격 필터링



## 6. Time Zone: `tz_localize`, `tz_convert`

타임존(시간대) 정보가 없는 naive datetime과, 타임존이 붙은 aware datetime을 구분하는 것이 중요합니다.


In [120]:
s = pd.to_datetime(pd.Series(['2024-01-01 12:00', '2024-01-02 12:00']))
s

0   2024-01-01 12:00:00
1   2024-01-02 12:00:00
dtype: datetime64[ns]

In [121]:
# 타임존 레이블 붙이기 (KST)
s_kst = s.dt.tz_localize('Asia/Seoul')
s_kst

0   2024-01-01 12:00:00+09:00
1   2024-01-02 12:00:00+09:00
dtype: datetime64[ns, Asia/Seoul]

In [122]:
# 다른 타임존으로 변환 (UTC)
s_utc = s_kst.dt.tz_convert('UTC')
s_utc

0   2024-01-01 03:00:00+00:00
1   2024-01-02 03:00:00+00:00
dtype: datetime64[ns, UTC]

- IANA TZDB 이름(권장): "Asia/Seoul", "America/New_York" 같은 Region/City 형태
  - Python zoneinfo도 이 IANA DB를 기준으로 동작
  - pandas tz_localize / tz_convert도 tz를 str로 받음

In [123]:
from zoneinfo import available_timezones
len(available_timezones())          # 내 환경에 있는 IANA tz 이름 개수
list(sorted(available_timezones()))[:20]  # 앞 20개 보기

['Africa/Abidjan',
 'Africa/Accra',
 'Africa/Addis_Ababa',
 'Africa/Algiers',
 'Africa/Asmara',
 'Africa/Asmera',
 'Africa/Bamako',
 'Africa/Bangui',
 'Africa/Banjul',
 'Africa/Bissau',
 'Africa/Blantyre',
 'Africa/Brazzaville',
 'Africa/Bujumbura',
 'Africa/Cairo',
 'Africa/Casablanca',
 'Africa/Ceuta',
 'Africa/Conakry',
 'Africa/Dakar',
 'Africa/Dar_es_Salaam',
 'Africa/Djibouti']

#### ✅ 추가 실습 6

1. 임의의 시각 시리즈를 만들고, 다양한 타임존(`Asia/Seoul`, `UTC`, `US/Eastern` 등)으로 변환해 보세요.
2. 타임존이 있는 시리즈와 없는 시리즈를 더하거나 비교할 때 어떤 오류/경고가 나는지 확인해 보세요.

> 아래 셀에서 직접 실습해 보세요.


In [124]:
# TODO: 추가 실습 6을 이 셀에서 진행해 보세요.

# 1) 여러 타임존 변환 실험
# 2) 타임존 유무에 따른 연산/비교 실험



## 7. NaT: 시간형의 결측값

숫자 타입에서 결측을 `NaN`으로 표현하듯,  
날짜/시간 타입에서 결측은 **`NaT (Not A Time)`**로 표현합니다.


In [125]:
s = pd.to_datetime(pd.Series(['2024-01-01', None, '2024-01-03']))
print(s)
print("dtype:", s.dtype)

0   2024-01-01
1          NaT
2   2024-01-03
dtype: datetime64[ns]
dtype: datetime64[ns]


In [126]:
# NaT가 포함된 상태에서 연산해 보기
s_plus_1d = s + pd.Timedelta(days=1)
s_diff = s.diff()

print("하루 더한 결과:")
print(s_plus_1d)

print("\n차이(diff) 결과:")
print(s_diff)

하루 더한 결과:
0   2024-01-02
1          NaT
2   2024-01-04
dtype: datetime64[ns]

차이(diff) 결과:
0   NaT
1   NaT
2   NaT
dtype: timedelta64[ns]


#### ✅ 추가 실습 7

1. `NaT`가 섞여 있는 시리즈에서, `dropna()`와 `fillna()`를 사용해 결측 처리해 보세요.  
2. `NaT`를 특정 기준 날짜(예: 1900-01-01)로 대체했을 때, 분석 결과가 어떻게 달라질 수 있는지 고민해 보세요.

> 아래 셀에서 직접 실습해 보세요.


In [127]:
# TODO: 추가 실습 7을 이 셀에서 진행해 보세요.

# 1) NaT 포함 시리즈 생성
# 2) dropna, fillna로 결측 처리



---

## 마무리

이 노트북에서 정리한 핵심 포인트:

- **`Timestamp` / `DatetimeIndex` / `datetime64[ns]`**: 시점(날짜/시간)
- **`Timedelta` / `TimedeltaIndex` / `timedelta64[ns]`**: 기간(시간 차, 경과 시간)
- **`DateOffset` 계열**: 달력/영업일 규칙 기반 이동 (월말, 분기말, 영업일 등)
- **`Period` / `PeriodIndex`**: 월·분기·연 같은 구간 단위 분석
- **`pd.to_datetime`, `pd.to_timedelta`, `.dt`**: 문자열/숫자 → 시간 타입 변환 및 파생 변수
- **Time Zone, NaT**: 실전 데이터에서 자주 마주치는 이슈들